<a href="https://colab.research.google.com/github/aletbm/MySolutions_StockMarketsAnalytics2025_DataTalks.Club/blob/main/02_Working_with_Data_in_Pandas/02_Working_with_Data_in_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Module 2 Homework (2025 Cohort)

In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the [LINK](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/02-dataframe-analysis/%5B2025%5D_Module_02_Colab_Working_with_the_data.ipynb) covered at the livestream to re-use the code snippets.

---
### Question 1: [IPO] Withdrawn IPOs by Company Type

**What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**

From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn](https://stockanalysis.com/ipos/withdrawn/)), collect and process the data to find out which company type saw the most withdrawn IPO value.

#### Steps:
1. Use `pandas.read_html()` with the URL above to load the IPO withdrawal table into a DataFrame.
   *It is a similar process to Code Snippet 1 discussed at the livestream.*    You should get **99 entries**.

In [1]:
import pandas as pd
import requests
from io import StringIO

def get_html(url):
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()

    # Wrap HTML text in StringIO to avoid deprecation warning
    # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
    html_io = StringIO(response.text)
    return html_io

url = f"https://stockanalysis.com/ipos/withdrawn"
html_io = get_html(url)
df = pd.read_html(html_io)[0]
df.head()

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


2. Create a new column called `Company Class`, categorizing company names based on patterns like:
   - “Acquisition Corp” or “Acquisition Corporation” → `Acq.Corp`
   - “Inc” or “Incorporated” → `Inc`
   - “Group” → `Group`
   - “Ltd” or “Limited” → `Limited`
   - “Holdings” → `Holdings`
   - Others → `Other`

  *  Order: Please follow the listed order of classes and assign the first matched value (e.g., for 'shenni holdings limited', you assign the 'Limited' class).

  * Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.

In [2]:
import re

def classify_company(value):
    value = value.lower()
    value = re.sub(r'[^a-z ]', "", value)
    words = value.split(" ")
    for word in words[::-1]:
        if word in ["acquisition", "corp"]:
            return "Acq.Corp"
        elif word in ["inc", "incorporated"]:
            return "Inc"
        elif word in ["group"]:
            return "Group"
        elif word in ["ltd", "limited"]:
            return "Limited"
        elif word in ["holdings"]:
            return "Holdings"
    return "Other"

df["company_class"] = df["Company Name"].apply(classify_company)
df.head()

,Symbol,Company Name,Price Range,Shares Offered,company_class
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc
2,AURN,"Aurion Biotech, Inc.",-,-,Inc
3,ROTR,"PHI Group, Inc.",-,-,Inc
4,ONE,One Power Company,-,-,Other


3. Define a new field `Avg. price` by parsing the `Price Range` field (create a function and apply it to the `Price Range` column). Examples:
   - '$8.00-$10.00' → `9.0`  
   - '$5.00' → `5.0`  
   - '-' → `None`

In [3]:
def parsing_price(price):
    price = re.sub('[^0-9 .]', "", price)
    price = price.replace("  ", " ")
    numbers = price.split(" ")

    if numbers[0] == '':
        return None

    avg = 0
    for number in numbers:
        avg += float(number)
    avg /= len(numbers)
    return avg

df["avg price"] = df["Price Range"].apply(parsing_price)
df.head()

,Symbol,Company Name,Price Range,Shares Offered,company_class,avg price
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,-,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,-,Inc,NaN
4,ONE,One Power Company,-,-,Other,NaN


4. Convert `Shares Offered` to numeric, clean missing or invalid values.

In [4]:
df["Shares Offered"] = df["Shares Offered"].apply(lambda x: None if x == "-" else x)
df["Shares Offered"] = df["Shares Offered"].astype(float)
df = df.dropna(subset=["Shares Offered"])
len(df)

72

5. Create a new column:  
   `Withdrawn Value = Shares Offered * Avg Price` (**71 non-null values**)

In [5]:
df["Withdrawn_millions"] = (df["Shares Offered"] * df["avg price"]) / 1_000_000
df.head()

/tmp/ipython-input-5-3851472107.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Withdrawn_millions"] = (df["Shares Offered"] * df["avg price"]) / 1_000_000


,Symbol,Company Name,Price Range,Shares Offered,company_class,avg price,Withdrawn_millions
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000.0,Inc,3.5,7.000
5,HPOT,The Great Restaurant Development Holdings Limited,$4.00 - $6.00,1400000.0,Limited,5.0,7.000
6,CABR,"Caring Brands, Inc.",$4.00,750000.0,Inc,4.0,3.000
7,SQVI,"Sequoia Vaccines, Inc.",$8.00 - $10.00,2775000.0,Inc,9.0,24.975
8,SNI,Shenni Holdings Limited,$4.00 - $6.00,3000000.0,Limited,5.0,15.000


6. Group by `Company Class` and calculate total withdrawn value.

In [6]:
df_withdrawn = df.groupby(by=["company_class"]).Withdrawn_millions.sum()
df_withdrawn

,Withdrawn_millions
company_class,
Acq.Corp,3936.849999
Group,27.187500
Inc,2257.164205
Limited,731.334585
Other,752.070000


7. **Answer**: Which class had the highest **total** value of withdrawals?

In [7]:
df_withdrawn.sort_values(ascending=False)

,Withdrawn_millions
company_class,
Acq.Corp,3936.849999
Inc,2257.164205
Other,752.070000
Limited,731.334585
Group,27.187500



---
### Question 2:   [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)


**What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?**

The goal is to replicate the large-scale `yfinance` OHLCV data download and perform basic financial calculations on IPO stocks.


#### Steps:

1. Using the same approach as in Question 1, download the IPOs in 2024 from:  
   [https://stockanalysis.com/ipos/2024/](https://stockanalysis.com/ipos/2024/)  
   Filter to keep only those IPOs **before 1 June 2024** (first 5 months of 2024).  
   ➤ You should have **75 tickers**.

In [8]:
url = f"https://stockanalysis.com/ipos/2024/"
html_io = get_html(url)
df_stocks = pd.read_html(html_io)[0]
df_stocks.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 31, 2024",ONEG,OneConstruction Group Limited,$4.00,$4.90,25.00%
1,"Dec 27, 2024",PHH,"Park Ha Biological Technology Co., Ltd.",$4.00,$28.21,564.25%
2,"Dec 23, 2024",HIT,"Health In Tech, Inc.",$4.00,$0.63,-84.40%
3,"Dec 23, 2024",TDAC,Translational Development Acquisition Corp.,$10.00,$10.27,2.80%
4,"Dec 20, 2024",RANG,Range Capital Acquisition Corp.,$10.00,$10.20,2.10%


In [9]:
df_stocks["IPO Price"] = df_stocks["IPO Price"].apply(lambda x: None if x == '-' else x)
df_stocks.dropna(subset=["IPO Price"], inplace=True)

df_stocks['IPO Date'] = pd.to_datetime(df_stocks['IPO Date'])
df_stocks = df_stocks.sort_values(by="IPO Date", ascending=True)
df_stocks = df_stocks.set_index("IPO Date")
df_stocks = df_stocks.loc[df_stocks.index < '2024-06-01']

len(df_stocks)

75

2.  Use **Code Snippet 7** to download daily stock data for those tickers (via `yfinance`).  
   Make sure you understand how `growth_1d` ... `growth_365d`, and volatility columns are defined.  
   Define a new column `growth_252d` representing growth after **252 trading days** (~1 year), in addition to any other growth periods you already track.

In [10]:
import time
import yfinance as yf
import numpy as np
from tqdm.notebook import tqdm

def get_historical_returns(tickers, trading_periods, future=False):
    stocks_df = pd.DataFrame({'A' : []})
    for i, ticker in tqdm(enumerate(tickers), total=len(tickers)):

        # Work with stock prices
        ticker_obj = yf.Ticker(ticker)
        historyPrices = ticker_obj.history(
                            period = "max",
                            interval = "1d")

        # generate features for historical prices, and what we want to predict
        historyPrices['Ticker'] = ticker
        historyPrices['Date'] = pd.to_datetime(historyPrices.index.date)

        # historical returns
        for i in trading_periods:
            if future:
                historyPrices[f'growth_future_{i}d'] = historyPrices['Close'].shift(-i) / historyPrices['Close']
            else:
                historyPrices[f'growth_{i}d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)

        # sleep 1 sec between downloads - not to overload the API server
        time.sleep(1)

        if stocks_df.empty:
            stocks_df = historyPrices
        else:
            stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

    return stocks_df

ALL_TICKERS = df_stocks['Symbol'].to_list()
stocks_df = get_historical_returns(tickers=ALL_TICKERS, trading_periods=[7, 30, 90, 252, 365])

  0%|          | 0/75 [00:00<?, ?it/s]

3. Calculate the Sharpe ratio assuming a risk-free rate of **4.5%**:

   ```python
   stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']
   ```

   ⚠️ **IMPORTANT** Please use the original version of annualized volatility calculation (it was later corrected to another formula):
   ```python
   stocks_df['volatility'] =   stocks_df['Close'].rolling(30).std() * np.sqrt(252)
   ```

In [11]:
stocks_df['volatility'] = stocks_df['Close'].rolling(30).std() * np.sqrt(252)
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']

4. Filter the DataFrame to keep data only for the trading day:  
   **‘2025-06-06’**

   Compute descriptive statistics (e.g., `.describe()`) for these columns:  
   - `growth_252d`  
   - `Sharpe`

   You should observe:  
   - `growth_252d` is defined for **71 out of 75 stocks** (some IPOs are too recent or data starts later).  
   - Median `growth_252d` is approximately **0.75** (indicating a 25% decline), while mean is about **1.15**, showing a bias towards high-growth companies pushing the average up.

In [12]:
filter_stocks_df = stocks_df[stocks_df.Date == '2025-06-06']
filter_stocks_df[["growth_252d", "Sharpe"]].describe().T

,count,mean,std,min,25%,50%,75%,max
growth_252d,71.0,1.152898,1.406017,0.024970,0.293422,0.758065,1.362736,8.097413
Sharpe,71.0,0.288285,0.519028,-0.079677,0.041215,0.083768,0.311507,2.835668


5. **Answer:**  
   - What is the **median Sharpe ratio** for these 71 stocks?  
   - Note: Positive `Sharpe` means growth exceeding the risk-free rate of 4.5%.  
   - [Additional] Do you observe the **same top 10 companies** when sorting by `growth_252d` versus sorting by `Sharpe`?

In [13]:
filter_stocks_df["Sharpe"].median()

0.0837682299507243

In [14]:
filter_stocks_df.sort_values(by="growth_252d", ascending=False).Ticker.head(10)

,Ticker
3176,JL
353,ROMA
9373,UMAC
20682,NNE
19285,RBRK
6981,AHR
5634,AS
18995,MRX
12128,RDDT
16651,MTEN


In [15]:
filter_stocks_df.sort_values(by="Sharpe", ascending=False).Ticker.head(10)

,Ticker
12719,BKHA
2468,JVSA
6637,LEGT
13317,IBAC
9034,HLXB
15462,MNDR
10009,DYCQ
11499,INTJ
3176,JL
17827,TRSG


---
### Question 3: [IPO] ‘Fixed Months Holding Strategy’

**What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?**  
(*Assume you buy at the close of the first trading day and sell after a fixed number of trading days.*)


---

#### Goal:
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

---

#### Steps:

1. **Start from the existing DataFrame** from Question 2 (75 tickers from IPOs in the first 5 months of 2024).  

   Add **12 new columns**:  
   `future_growth_1m`, `future_growth_2m`, ..., `future_growth_12m`  
   *(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)*  
   This logic is similar to `historyPrices['growth_future_30d']` from **Code Snippet 7**, but extended to longer timeframes.

In [16]:
trading_days = np.arange(0, 253, 21)[1:]
stocks_df = get_historical_returns(tickers=ALL_TICKERS, trading_periods=trading_days, future=True)
stocks_df = stocks_df.rename(columns={f'growth_future_{days}d': f'future_growth_{i+1}m' for i, days in enumerate(trading_days)})

  0%|          | 0/75 [00:00<?, ?it/s]

2. **Determine the first trading day** (`min_date`) for each ticker.  
   This is the earliest date in the data for each stock.

In [17]:
min_date = stocks_df.groupby(by=["Ticker"]).Date.min()
min_date.head()

,Date
Ticker,
AHR,2024-02-07
ALAB,2024-03-20
ANRO,2024-02-02
AS,2024-02-01
AUNA,2024-03-22


3. **Join the data**:  
   Perform an **inner join** between the `min_date` DataFrame and the future growth data on both `ticker` and `date`.  
   ➤ You should end up with **75 records** (one per IPO) with all 12 `future_growth_...` fields populated.

In [18]:
df = pd.merge(min_date, stocks_df, on=['Ticker', 'Date'], how='inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Ticker             75 non-null     object        
 1   Date               75 non-null     datetime64[ns]
 2   Open               75 non-null     float64       
 3   High               75 non-null     float64       
 4   Low                75 non-null     float64       
 5   Close              75 non-null     float64       
 6   Volume             75 non-null     int64         
 7   Dividends          75 non-null     float64       
 8   Stock Splits       75 non-null     float64       
 9   future_growth_1m   75 non-null     float64       
 10  future_growth_2m   75 non-null     float64       
 11  future_growth_3m   75 non-null     float64       
 12  future_growth_4m   75 non-null     float64       
 13  future_growth_5m   75 non-null     float64       
 14  future_growt

4. **Compute descriptive statistics** for the resulting DataFrame:  
   Use `.describe()` or similar to analyze each of the 12 columns:  
   - `future_growth_1m`  
   - `future_growth_2m`  
   - ...  
   - `future_growth_12m`  


In [19]:
df.describe().T[8:]

,count,mean,min,25%,50%,75%,max,std
future_growth_1m,75.0,0.927259,0.098947,0.778984,0.977,1.046509,2.646505,0.346261
future_growth_2m,75.0,0.940544,0.0738,0.685815,1.0,1.154013,4.874759,0.574545
future_growth_3m,75.0,0.833825,0.060947,0.511212,0.9275,1.069085,2.04,0.409763
future_growth_4m,75.0,0.825087,0.045368,0.517233,0.909091,1.1343,1.605,0.401969
future_growth_5m,75.0,0.803769,0.054109,0.448403,0.821092,1.016381,3.213873,0.488349
future_growth_6m,75.0,0.864186,0.061432,0.38456,0.802239,1.093948,3.67052,0.65318
future_growth_7m,75.0,0.84715,0.044086,0.29687,0.844875,1.114468,5.12235,0.712944
future_growth_8m,75.0,0.832984,0.043103,0.208677,0.812109,1.082457,5.171484,0.762423
future_growth_9m,75.0,0.88178,0.033144,0.22674,0.822715,1.049719,6.764933,0.936957
future_growth_10m,74.0,0.917946,0.037769,0.242424,0.772592,1.20083,5.352601,0.911431


5. **Determine the best holding period**:  
   - Find the number of months **(1 to 12)** where the **average (mean)** future growth is **maximal**.  
   - This optimal month shows an uplift of **>1%** compared to all others.  
   - Still, the average return remains **less than 1** (i.e., expected return is less than doubling your investment).



In [20]:
describe = df.describe().T[8:].sort_values(by="mean", ascending=False)[0:2]
describe

,count,mean,min,25%,50%,75%,max,std
future_growth_2m,75.0,0.940544,0.0738,0.685815,1.0,1.154013,4.874759,0.574545
future_growth_1m,75.0,0.927259,0.098947,0.778984,0.977,1.046509,2.646505,0.346261


In [21]:
per = (describe.loc[describe.index[0], "mean"] - describe.loc[describe.index[1], "mean"]) * 100 / describe.loc[describe.index[1], "mean"]
print(f"Uplift of {round(per, 2)}%")

Uplift of 1.43%


In [22]:
describe.loc[describe.index[0], "mean"] < 1

True

---
### Question 4: [Strategy] Simple RSI-Based Trading Strategy


**What is the total profit (in \$thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?**


---

#### Goal:
Apply a simple rule-based trading strategy using the **Relative Strength Index (RSI)** technical indicator to identify oversold signals and calculate profits.

---


#### Steps:

1. **Run the full notebook from Lecture 2 (33 stocks)**  
   - Ensure you can generate the merged DataFrame containing:  
     - OHLCV data  
     - Technical indicators  
     - Macro indicators  
   - Focus on getting **RSI** computed using **Code Snippets 8 and 9**.  
   - This process is essential and will help during the capstone project.


2. ⚠️ **IMPORTANT** Please use this file to solve the Home Assignment (**all next steps**)

   Download precomputed data using this snippet:

   ```python
   import gdown
   import pandas as pd

   file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
   gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
   df = pd.read_parquet("data.parquet", engine="pyarrow")
   ```

In [23]:
import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")
df.head()

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=13789ae0-4126-4eee-8066-452339335494
To: /content/data.parquet
100%|██████████| 130M/130M [00:01<00:00, 83.9MB/s]


,Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
0,0.054277,0.062259,0.054277,0.059598,1.031789e+09,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.059598,0.062791,0.059598,0.061726,3.081600e+08,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.061726,0.063323,0.061726,0.062791,1.331712e+08,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.062791,0.063323,0.060662,0.061194,6.776640e+07,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.061194,0.061726,0.059598,0.060130,4.789440e+07,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


3. **RSI Strategy Setup:**  
   - RSI is already available in the dataset as a field.  
   - The threshold for **oversold** is defined as `RSI < 25`.

In [24]:
df[df.rsi < 25].rsi

,rsi
608,24.986214
609,22.563828
610,20.757761
1112,24.757850
1122,24.403145
...,...
4405,14.881342
4406,19.198155
4407,14.161711
4408,13.986511


4. **Filter the dataset by RSI and date:**  
   ```python
   rsi_threshold = 25
   selected_df = df[
       (df['rsi'] < rsi_threshold) &
       (df['Date'] >= '2000-01-01') &
       (df['Date'] <= '2025-06-01')
   ]
   ```


In [25]:
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]
selected_df.head()

,Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
3668,20.056772,20.114241,19.405453,19.673643,99915200.0,0.0,0.0,MSFT,2000,2000-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3669,19.692798,19.807736,19.060636,19.309669,69037800.0,0.0,0.0,MSFT,2000,2000-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3680,18.256067,18.332693,17.317403,17.336559,85374000.0,0.0,0.0,MSFT,2000,2000-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3681,17.279087,17.336557,16.704395,16.991741,136453400.0,0.0,0.0,MSFT,2000,2000-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3682,17.010902,17.547282,16.934277,16.972589,81099400.0,0.0,0.0,MSFT,2000,2000-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


5. **Calculate Net Profit Over 25 Years:**  
   - Total number of trades: **1568**  
   - For each trade, you invest **$1000**  
   - Use the 30-day forward return (`growth_future_30d`) to compute net earnings:  
     ```python
     net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
     ```

In [28]:
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()

   - **Final Answer:**  
     What is the **net income in $K** (i.e., in thousands of dollars) that could be earned using this RSI-based oversold strategy from 2000–2025?

In [36]:
float(net_income / 1_000)

24.295523125248387

---
### Q5. [Exploratory, Optional] Predicting a Positive-Return IPO

Most of the strategies for investing in IPOs deliver **negative average and median returns** (and even 75% quantiles).

**Question:**  
How would you change the strategy if you want to **increase the profitability**?

> This is an open-ended brainstorming question — propose ideas for identifying IPOs with positive future returns or building a more effective trading strategy.

## “Wait-and-Score” Strategy (Post-IPO Selection and Timing)

> Most IPO investment strategies deliver negative average and median returns because they are often driven by hype, overvaluation, and early insider selling. To increase profitability, I would shift away from buying on IPO day and instead adopt a "wait-and-score" strategy.
This strategy involves waiting 3 to 6 months after the IPO before considering an investment. This period allows the market to absorb early volatility, insider lock-up expirations, and initial earnings reports. Once this window has passed, I would screen IPOs based on a combination of fundamental and technical factors.
Specifically, I would target companies that demonstrate strong fundamentals (such as profitability or a clear path to it), positive momentum (price trading above the IPO price and key moving averages), and signs of insider confidence (high insider ownership and limited selling). Additional filters could include positive earnings surprises, reasonable valuations compared to peers, and underwriters with strong reputations.
Rather than betting on individual names, I would build a diversified basket of high-scoring IPOs and apply strict risk management — including position sizing and stop-loss rules — to control downside.
By avoiding the initial hype phase and focusing on quality and momentum, this approach increases the likelihood of capturing the long-term winners among IPOs while avoiding the majority that underperform.